# 智谱API测试

## 使用 ChatZhipuAI

In [6]:
!pip install pydantic==1.10.13
# !pip install pydantic==2.6.2

  Using cached pydantic-1.10.13-cp39-cp39-macosx_10_9_x86_64.whl.metadata (149 kB)
Using cached pydantic-1.10.13-cp39-cp39-macosx_10_9_x86_64.whl (2.9 MB)
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.6.2
    Uninstalling pydantic-2.6.2:
      Successfully uninstalled pydantic-2.6.2


In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [2]:
from langchain_chinese import ChatZhipuAI
from langchain_core.prompts import ChatPromptTemplate

In [3]:
llm = ChatZhipuAI()

In [4]:
llm.invoke("hi")

AIMessage(content='你好👋！我是人工智能助手智谱清言（ChatGLM），很高兴见到你，欢迎问我任何问题。')

In [ ]:
await llm.ainvoke("hi")

AIMessage(content='你好👋！我是人工智能助手智谱清言（ChatGLM），很高兴见到你，欢迎问我任何问题。')

In [5]:
for chunk in llm.stream("hi"):
  print(chunk.content, end="|", flush=True)

Hello| |👋!| I|'|m| Chat|GL|M|（|智|谱|清|言|）,| the| artificial| intelligence| assistant|,| nice| to| meet| you|.| Feel| free| to| ask| me| any| questions|.||

In [8]:
async for chunk in llm.astream("hi"):
  print(chunk.content, end="|", flush=True)

Hello|!| How| can| I| assist| you| today|?||

In [10]:
async for chunk in llm.astream_events("hi", version="v1"):
  print(chunk)

{'event': 'on_chat_model_start', 'run_id': '4f202b64-1034-455b-9bc4-5fc32e027565', 'name': 'ChatZhipuAI', 'tags': [], 'metadata': {}, 'data': {'input': 'hi'}}
{'event': 'on_chat_model_stream', 'run_id': '4f202b64-1034-455b-9bc4-5fc32e027565', 'tags': [], 'metadata': {}, 'name': 'ChatZhipuAI', 'data': {'chunk': AIMessageChunk(content='你好')}}
{'event': 'on_chat_model_stream', 'run_id': '4f202b64-1034-455b-9bc4-5fc32e027565', 'tags': [], 'metadata': {}, 'name': 'ChatZhipuAI', 'data': {'chunk': AIMessageChunk(content='👋！')}}
{'event': 'on_chat_model_stream', 'run_id': '4f202b64-1034-455b-9bc4-5fc32e027565', 'tags': [], 'metadata': {}, 'name': 'ChatZhipuAI', 'data': {'chunk': AIMessageChunk(content='我是')}}
{'event': 'on_chat_model_stream', 'run_id': '4f202b64-1034-455b-9bc4-5fc32e027565', 'tags': [], 'metadata': {}, 'name': 'ChatZhipuAI', 'data': {'chunk': AIMessageChunk(content='人工智能')}}
{'event': 'on_chat_model_stream', 'run_id': '4f202b64-1034-455b-9bc4-5fc32e027565', 'tags': [], 'metada

In [6]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "query_train_info",
            "description": "根据用户提供的信息，查询对应的车次",
            "parameters": {
                "type": "object",
                "properties": {
                    "departure": {
                        "type": "string",
                        "description": "出发城市或车站",
                    },
                    "destination": {
                        "type": "string",
                        "description": "目的地城市或车站",
                    },
                    "date": {
                        "type": "string",
                        "description": "要查询的车次日期",
                    },
                },
                "required": ["departure", "destination", "date"],
            },
        }
    }
]

In [7]:
llm = ChatZhipuAI().bind(tools=tools)

In [8]:
llm

RunnableBinding(bound=ChatZhipuAI(client=<zhipuai._client.ZhipuAI object at 0x110ca1820>), kwargs={'tools': [{'type': 'function', 'function': {'name': 'query_train_info', 'description': '根据用户提供的信息，查询对应的车次', 'parameters': {'type': 'object', 'properties': {'departure': {'type': 'string', 'description': '出发城市或车站'}, 'destination': {'type': 'string', 'description': '目的地城市或车站'}, 'date': {'type': 'string', 'description': '要查询的车次日期'}}, 'required': ['departure', 'destination', 'date']}}}]})

In [9]:
llm.invoke("给我讲个一句话笑话")

AIMessage(content='一只羊说：“我每天都比一只狼跑得快，因为我比它更会绕圈子。”')

In [10]:
llm.invoke("你能帮我查询2024年1月1日从北京南站到上海的火车票吗？")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8415022186389879829', 'function': {'arguments': '{"date":"2024-01-01","departure":"北京南站","destination":"上海"}', 'name': 'query_train_info'}, 'type': 'function'}]})

In [ ]:
for chunk in llm.stream("你能帮我查询2024年1月1日从北京南站到上海的火车票吗？"):
  print(chunk)

content='' additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_8367745969533447732', 'function': {'arguments': '{"date":"2024-01-01","departure":"北京南站","destination":"上海"}', 'name': 'query_train_info'}, 'type': 'function'}]}
content=''


In [ ]:
llm = ChatZhipuAI()
llm.invoke("贾玲最新的作品是什么？")

AIMessage(content='根据提供的信息，贾玲的最新作品是《热辣滚烫》，这部电影在春节档期上映，并且取得了相当的成功，票房超过23.6亿元，豆瓣评分达到了8.0分，是当时春节档期评分最高的电影。贾玲不仅是该片的导演，也通过该片获得了观众和市场的广泛认可。\n\n此外，贾玲还透露了她的下一部作品的一些信息。她的新片名为《转念花开》（又名《光明之战》），是一部关于反传销题材的电影。这部电影已经备案，并且贾玲表示已经拍摄了一部分，剩下的一部分还在拍摄计划中。贾玲还提到，她已经邀请杨紫参与新片的演出，这增加了观众对这部电影的期待。')

In [ ]:
tools = [
    {
        "type": "web_search",
        "web_search": {
            "search_query": "贾玲演过的相声"
        }
    }
]
llm = ChatZhipuAI().bind(tools=tools)
llm.invoke("列举一个贾玲的作品")

AIMessage(content='贾玲是中国内地的著名女演员、喜剧演员和制片人，她有许多知名的作品。其中，贾玲最著名的作品之一是《欢乐喜剧人》。这是一档喜剧竞技类节目，贾玲作为固定成员和队长，带领团队在节目中表演了许多精彩的喜剧作品，深受观众喜爱。通过这个节目，贾玲展示了自己出色的喜剧表演才华，赢得了广泛的赞誉和人气。')

In [ ]:
tools = [
    {
        "type": "web_search",
        "web_search": {
            "search_query": "贾玲演过的电影"
        }
    }
]
llm = ChatZhipuAI().bind(tools=tools)
llm.invoke("从背景信息中找出贾玲的作品")

AIMessage(content='很抱歉，您没有提供具体的背景信息或者说明，所以我无法直接指出贾玲的作品。贾玲是中国的一位著名女喜剧演员、导演和制片人。如果您能提供更多的上下文或细节，比如关于某个特定节目、事件或者作品的信息，我将能够更好地帮助您找出贾玲的相关作品。例如，贾玲因参与《欢乐喜剧人》等节目以及导演和参与电影《你好，李焕英》而广受欢迎。提供更多细节后，我可以为您提供更精确的答案。')